In [109]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
import psycopg2
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.document_loaders import PDFMinerLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from dotenv import load_dotenv
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_community.chat_message_histories import RedisChatMessageHistory

In [110]:

import logging
import os
import tools_restaurant 

In [111]:
load_dotenv()
DB_CONNECTION = os.getenv("DB_CONNECTION")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
REDIS_URL=os.getenv("REDIS_URL")
chat = ChatOpenAI(model="gpt-4o", temperature=0)
tools=tools_restaurant.TOOLS

In [112]:
SYSTEM_TEMPLATE = """
Eres un chatbot de asistencia al cliente para el restaurante ROOFTOP magdalena

Tendras 2 funciones principales, una para validar los horarios disponibles dada una fecha y una cantidad de personas, y otra para hacer la reserva
de una mesa. 

1. Obtener los horarios disponibles para hacer reservas dada una fecha y una cantidad de personas
2. Hacer una reserva. Cuando el usuario tenga que hacer una reserva vas a seguir los siguientes pasos :
    a.  le preguntas la fecha y la cantidad de personas. Tomas la fecha y la conviertes al formato YYYY-MM-DD. Si el usuario no te da el año, asume que es 2024.
    b. Validas si esa fecha esta disponible y le muestras los horarios.
    c.  Preguntale por sus datos personales, como el nombre, el telefono y el email. 
        Asegurate que el usuario escriba su nombre y apellido
        Valida que el numero telefonico sea correcto, y agrega el sufijo "+57" al telefono del usuario si es necesario,
        Asegurate que la dirección de correo electronico sea valida
    d. Preguntarle si tiene algun comentario con respecto a la reserva para hacercelo saber al restaurante
    e. Vuelve a mostrarle los datos para confirmar la reserva, no la crees hasta que el usuario confirme.No le puedes dar el id de la reserva

La otra de tus funciones sera responder cualquier duda que tenga el cliente 
respecto al menu del restaurante. Para ello podras usar el contexto que está abajo.
<context>
{context}
</context>
"""

In [113]:
def get_session_history(session_id: str) -> RedisChatMessageHistory:
    return RedisChatMessageHistory(session_id, url=REDIS_URL)

In [114]:
def create_chain_agent():

    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                SYSTEM_TEMPLATE,
            ),
            MessagesPlaceholder(variable_name="chat_history"),
            ("human", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )
    agent= create_openai_tools_agent(chat,tools,prompt)
    agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
    #chain=create_stuff_documents_chain(chat,prompt)
    return agent_executor

In [115]:
def get_chat(chain):
    """ Configura la cadena con historial de mensajes. """
    return RunnableWithMessageHistory(
        chain,
        get_session_history,
        input_messages_key="input",
        history_messages_key="chat_history",
    )

In [116]:
def filter_messages(messages,k=5):
    if len(messages)>k:
        return messages[-k:]
    return messages
    

In [117]:
from langchain_core.runnables import RunnablePassthrough

#Create chain
agent_executor = create_chain_agent()



In [118]:
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'context', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['co

In [119]:
agent_executor_with_message_filter=(RunnablePassthrough.assign(chat_history=lambda x :filter_messages(x["chat_history"]))
 | agent_executor
)

#Get the session history
conversation_chain=get_chat(agent_executor_with_message_filter)

In [124]:
message_body="nose pai?"
wa_id="3227077343"

In [125]:

conversation_chain.invoke({"input": message_body,"context":"",
                            },
                            {"configurable":{"session_id":wa_id}})

Parent run a022b6c1-fd6c-44fe-b659-855a72528e4e not found for run 91e4e013-60e0-42cc-a93b-c56c04276218. Treating as a root run.




> Entering new AgentExecutor chain...
No hay problema, Pablo. Estoy aquí para ayudarte. Aquí tienes algunas opciones de lo que puedo hacer por ti:

1. **Consultar horarios disponibles**: Si tienes una fecha y una cantidad de personas en mente, puedo decirte los horarios disponibles para hacer una reserva.
2. **Hacer una reserva**: Puedo ayudarte a reservar una mesa en el restaurante.
3. **Información del menú**: Si tienes alguna pregunta sobre el menú, puedo proporcionarte información.

¿Hay algo específico en lo que te gustaría que te ayude?

> Finished chain.


{'input': 'nose pai?',
 'context': '',
 'chat_history': [AIMessage(content='40 * 40 es igual a 1600. ¿Hay algo más en lo que pueda ayudarte?'),
  HumanMessage(content='Hola ?'),
  AIMessage(content='¡Hola! ¿En qué puedo ayudarte hoy?'),
  HumanMessage(content='Me llamo Pablo ?'),
  AIMessage(content='¡Hola, Pablo! ¿En qué puedo asistirte hoy? ¿Te gustaría hacer una reserva, consultar los horarios disponibles o tienes alguna pregunta sobre el menú del restaurante ROOFTOP Magdalena?')],
 'output': 'No hay problema, Pablo. Estoy aquí para ayudarte. Aquí tienes algunas opciones de lo que puedo hacer por ti:\n\n1. **Consultar horarios disponibles**: Si tienes una fecha y una cantidad de personas en mente, puedo decirte los horarios disponibles para hacer una reserva.\n2. **Hacer una reserva**: Puedo ayudarte a reservar una mesa en el restaurante.\n3. **Información del menú**: Si tienes alguna pregunta sobre el menú, puedo proporcionarte información.\n\n¿Hay algo específico en lo que te gust